In [ ]:
!pip install imutils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -qq '/content/drive/MyDrive/HandWritten_Character.zip'

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import random 
import cv2
import imutils
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from keras.utils import np_utils
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow import keras

In [ ]:
file_path1 = '/content/Train'

train = []
img_size = 32
non_chars = ['#', '$', '&', '@']

for i in os.listdir(file_path1):
  if i in non_chars:
        continue
  count = 0
  sub_directory = os.path.join(file_path1, i)
  
  for j in os.listdir(sub_directory):
    count+=1

    if count > 4000:
      break
    
    img = cv2.imread(os.path.join(sub_directory, j),0)
    img = cv2.resize(img,(img_size, img_size))
    train.append([img, i])

len(train)

In [ ]:
file_path2 = '/content/Validation'

test = []
img_size = 32

for i in os.listdir(file_path2):
  if i in non_chars:
        continue
  count = 0
  sub_directory1 = os.path.join(file_path2, i)
  
  for j in os.listdir(sub_directory1):
    count+=1

    if count > 1000:
      break

    img1 = cv2.imread(os.path.join(sub_directory1, j),0)
    img1 = cv2.resize(img1,(img_size, img_size))
    test.append([img1, i])

len(test)

In [ ]:
random.shuffle(train)
random.shuffle(test)

In [ ]:
X_train = []
y_train = []

for features, label in train:
    X_train.append(features)
    y_train.append(label)

X_test = []
y_test = []

for features, label in test:
    X_test.append(features)
    y_test.append(label)

In [ ]:
lbb = LabelBinarizer() # 라벨을 일 대 다 형식으로 이진화 한다 / 이는 라벨당 라벨인지 아닌지에 대한 이진분류와 비슷한 개념으로 설명된다

y_train = lbb.fit_transform(y_train)
y_test = lbb.fit_transform(y_test)

In [ ]:
X_train = np.array(X_train) / 255.0
X_train = X_train.reshape(-1, 32, 32, 1)
y_train = np.array(y_train)

X_test = np.array(X_test) / 255.0
X_test = X_test.reshape(-1, 32, 32, 1)
y_test = np.array(y_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(32, 3, padding = 'same', activation = 'relu', input_shape = (32, 32, 1)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(64, 3, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128, 3, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4)) 
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(35, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes = True)

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])

In [ ]:
line = model.fit(X_train, y_train, epochs = 30, batch_size = 32, validation_data = (X_test, y_test), verbose = 1)

In [ ]:
plt.plot(line.history['accuracy'])
plt.plot(line.history['val_accuracy'])
plt.title('Training Accuracy vs Val Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.show()

에포크를 5 ~ 10 사이에서 끊어도 성능에는 큰 변화가 없다는 것을 알 수 있다

In [ ]:
def sort_contours(cnts, method = 'left-to-right'):
  
  reverse = False
  
  i = 0
    
  if method == 'right-to-left' or method == 'bottom-to-top':
      reverse = True
    
  if method == 'top-to-bottom' or method == 'bottom-to-top':
      i = 1
    
  boundingBoxes = [cv2.boundingRect(c) for c in cnts]
  (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
  key=lambda b:b[1][i], reverse=reverse))

  return (cnts, boundingBoxes)

def get_letters(img):
    
  letters = []
  image = cv2.imread(img)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  ret, thresh1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
  dilated = cv2.dilate(thresh1, None, iterations = 2)

  cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  cnts = sort_contours(cnts, method = 'left-to-right')[0]

  for c in cnts:
    
    if cv2.contourArea(c) > 10:
      (x, y, w, h) = cv2.boundingRect(c)
      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
      
    roi = gray[y : y + h, x : x + w]
    thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    thresh = cv2.resize(thresh, (32, 32), interpolation = cv2.INTER_CUBIC)
    thresh = thresh.astype('float32') / 255.0
    thresh = np.expand_dims(thresh, axis = -1)
    thresh = thresh.reshape(1, 32, 32, 1)
    ypred = model.predict(thresh)
    ypred = lbb.inverse_transform(ypred)
    [x] = ypred
    letters.append(x)

  return letters, image

def get_word(letter):
  
  word = ''.join(letter)
  
  return word

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
letter,image = get_letters('TEST_0193.jpg')
word = get_word(letter)

print(word)
plt.imshow(image)

In [ ]:
import joblib

joblib.dump(model, "ocr.pkl")